In [6]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [13]:
using LinearAlgebra
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using PushRecovery
BLAS.set_num_threads(1)

In [9]:
@time AtlasRobot.mechanism(add_flat_ground=true);

  7.765894 seconds (20.88 M allocations: 1.052 GiB, 7.04% gc time)


In [10]:
@time mechanism = AtlasRobot.mechanism(add_flat_ground=true);

  0.008467 seconds (30.01 k allocations: 1.459 MiB)


In [30]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = point.model.friction.μ
            contact = addcontact!(lowlevel, body, position, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [31]:
# create high level controller
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)
controller = let
    atol_distance = 1e-3
    convexhulloptimizer = OSQP.Optimizer()
    MOI.set(convexhulloptimizer, OSQPSettings.Verbose(), false)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsAbs(), atol_distance^2)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsRel(), 1e-8)
#     MOI.set(convexhulloptimizer, OSQPSettings.Polish(), true)
#     MOI.set(convexhulloptimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    PushRecoveryController(lowlevel, feet, pelvis, nominalstate, convexhulloptimizer)
end;

In [32]:
# create push applier
pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0));

In [33]:
# create visualizer
using MeshCat
using MeshCatMechanisms

if !(@isdefined gui) || !any(isopen, gui.visualizer.visualizer.core.scope.pool.connections)
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    mvis = MechanismVisualizer(mechanism, visuals)
    gui = GUI(mvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
end

set_configuration!(gui.visualizer, configuration(nominalstate))

In [46]:
# create ODEProblem
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., Inf); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, Inf)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [48]:
# simulate
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
last(sol.t)

 74.726553 seconds (76.46 M allocations: 4.546 GiB, 6.17% gc time)


73.8256923926877

In [40]:
setanimation!(mvis, sol)

In [13]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult